## Computing the Distance between Two Points
*Draft* exercise, last updated 11/16/2015
pattyf@berkeley.edu

### Introduction

Most of us learned in high school that we can use the Pythagorean Theorem to calculate the straight-line distance between two points. The theorem can be written as an equation that relates the 3 sides of a triangle:

**a2 + b2 = c2**.

In other words, given point (x1,y1) and point (x2,y2) the distance between them is equal to the square root of sum of  (x1 - x2) and (y1 - y2).  This is known as the **Euclidean distance**.

Let's use this method to calculate the distance between two international airports, Seattle Tacoma International Airport (SEA) and Dallas / Fort Worth International Airport (DFW). We can get the geographic coordinates for these places via a search http://geonames.org. The following coordinates, in latitude-longitude order, were retrieved:

- 47.449, -122.309 # Seattle Tacoma International Airport (SEA)
- 32.896, -97.037 # Dallas / Fort Worth International Airport (DFW)

The distance between these two airports, according to the Great Circle Mapper (http://www.gcmap.com/) is **1660** miles or **2671.5** kilometers (km). Let's figure out the best method to calculate an accurate distance between these locations and why.

#### References

- http://geonames.org
- https://en.wikipedia.org/wiki/Longitude
- https://en.wikipedia.org/wiki/Latitude
- http://maps.google.com
- https://source.opennews.org/en-US/learning/choosing-right-map-projection/
- https://en.wikipedia.org/wiki/Pythagorean_theorem


In [18]:
import math
from geopy.distance import vincenty

In [19]:
SEA = (47.449, -122.309) # Seattle Tacoma International Airport (SEA)
DFW = (32.896, -97.037) # Dallas / Fort Worth International Airport (DFW)

In [20]:
# a function to calculate distance using the pythagorean theorem
def euclidean_dist ( x1, y1, x2, y2 ):
    x_dist = x1 - x2  
    y_dist = y1-y2    
    dist_sq = x_dist**2 + y_dist**2   

    distance = math.sqrt(dist_sq)   
    return distance

In [21]:
euclidean_dist(SEA[1],SEA[0],DFW[1], DFW[0])

29.162712373851637

### Interpretting the output

What does that result mean? What are are units? Is that the correct distance?

The value 29 is no where near the distance give to us by GCM (2671.5 km). The units are in decimal degrees which are angular units not linear. One might be tempted to convert decimal degrees to kilometers thinking they both measure distance on the surface of the earth so there must be a conversion factor. Well, because the earth is not flat it's complicated:

- 1 degree of latitude on the earth measures about 111.2 km or 69 miles. It varies very slightly from 110.6 km at equator to 111.7 km at the poles).
- 1 degree of longitude varies from 111.3 km  at the equator to 0 at the poles.


### Questions

    - Given the above information, write a formula that estimates the circumference of the earth
        - at the equator
        - at the poles
    - Which value is larger and why?
  

Let's simplify and assume that one degree of latitude or longitude is about 111 KM and convert the distance in degrees to kilometers.


In [22]:
euclidean_dist(SEA[1],SEA[0],DFW[1], DFW[0]) * 111


3237.0610734975317

Our distance calculation is now off by **565** km (3237.1 km - 2671.5 km). So, clearly we cannot assume a flat earth when we calculate geographic distances.

### Measuring Distance on the Spherical Earth

The simplest model of the non-flat earth is a sphere. Given a spherical model, we can calculate distances on the earth  if we know the circumference of the sphere. The shortest distance between two points along a sphere is called a great-circle diatance. The calculation employs the **haversine formula** and is shown below. Because of their mathematical properties of radians, angular measurements based on radians rather than degrees are used. (One radian is equal to 180/pi degrees.)

In [23]:
# a function to calculate distance on a sphere
EARTH_RADIUS_KM = 6371
def haversine_dist ( x1, y1, x2, y2 ):
    x_dist_h = math.radians(x1 - x2)
    y_dist_h = math.radians(y1 - y2)
    y1_rad = math.radians(y1)
    y2_rad = math.radians(y2)
    a = math.sin(y_dist_h/2)**2 + math.sin(x_dist_h/2)**2 * math.cos(y1_rad) * math.cos(y2_rad)
    c = 2 * math.asin(math.sqrt(a))
    distance_h = c * EARTH_RADIUS_KM
    return distance_h

In [24]:
print(haversine_dist(SEA[1],SEA[0],DFW[1], DFW[0]) )


2668.28220942


Using the spherical model, our distance is now only 3.5 km off of the Great Circle Mapper. That's pretty good. Why not just call it a day? There are two main reasons: 1) because some applications require a higher level of accuracy (e.g., a drone delivering your package) and  2) because the earth, as much as we would like to to be, is not a sphere.  Its shape is closer to an **ellipsoid** (which is sometimes refered to as a spheriod). We have only calculated the distance between two points on the earth. Distance calculations based on a spherical earth model may be much more accurate for other locations.

### Measuring Distance on the Ellipsoidal Earth

A ellipsoidal model of the earth is specified by the radius of the major and minor axes or by the radius of the major axis and the polar flattening ratio. The main reference ellipsoids in use in North America are:

- World Geodetic System of 1984, WGS-84 (6378.137,    6356.7523142)
- The Geodetic Reference system of 1980 GRS-80 (6378.137,    6356.7523141)
- Clarke (1880) (6378.249145, 6356.51486955) (now obsolete but data floating around that use it.)

WGS84 is the default ellipsoidal model and is implemented by the WGS84 Geographic Coordinate Reference System (CRS).

Distance calculataions based on the ellipsoidal model of the earth employ **Vincenty's formula**. This is much more complicated math so we will use it's implementation in the python **geopy** module. You can read about it here:
https://github.com/geopy/geopy/blob/master/geopy/distance.py

In [25]:
# try vincenty - default ellipsoid is wgs84,others listed at: 
# https://github.com/geopy/geopy/blob/master/geopy/distance.py
print(vincenty(SEA, DFW).km)

2671.13691103


We now have approximately the same distance using the vincenty formula as we got from Great Circle Mapper! That is because GCM uses this formula (see: http://www.gcmap.com/faq/gccalc#gchow0). 

### Question
Complete the code block below to compute the vincenty distance between these two airports using all three ellipsoids. See the url above for the function arguments. Discuss on any differences in the results.

In [26]:
### Answer:
#print(vincenty(..).km)
#print(vincenty(..).km)
#print(vincenty(..).km)


You can see from above that *for these locations* the WGS84 and GRS80 ellipsoids give us the same result. This may not be true for all locations on the earth so it is important to specify the ellipsoid or CRS that you use. 

But wait! If we can accurately calculate distances using the ellipsoidal earth model why do we need map projections?


### Map Projections

Map projections are used to transform 3D geographic coordinates to a 2D planar surface. We do this so we can produce 2D visualizations of geographic locations, e.g., printed maps and on computer screens. We also need map projections because ellipsoidal calculations more complex than point distances are too computationally intensive and are not widely implemented.  Map projections are designed to minimize distortion in area, shape, distance and/or direction for specific regions on the earth. No map projection can eliminate all forms of distortion for all regions. The selection of the appropriate map projection is extremely important and must be made with respect to the needs of the application at hand.



### We can use the gdal module to transform geographic coordinates to projected coordinates.

In order to transform geographic coordinates to projected coordinates we need to import some modules to help with the calculations. The osgeo gdal and ogr libraries are python ports of the powerful and widely used GDAL/OGR libraries for importing and transforming geospatial data. See http://gdal.org for details on its functionality.

In [27]:
from osgeo import ogr, osr, gdal

# Enable GDAL/OGR exceptions
gdal.UseExceptions()
# example GDAL error handler function

def gdal_error_handler(err_class, err_num, err_msg):
    errtype = {
            gdal.CE_None:'None',
            gdal.CE_Debug:'Debug',
            gdal.CE_Warning:'Warning',
            gdal.CE_Failure:'Failure',
            gdal.CE_Fatal:'Fatal'
    }
    err_msg = err_msg.replace('\n',' ')
    err_class = errtype.get(err_class, 'None')
    print 'Error Number: %s' % (err_num)
    print 'Error Type: %s' % (err_class)
    print 'Error Message: %s' % (err_msg)

gdal.PushErrorHandler(gdal_error_handler)

0

In [28]:
#gdal is having trouble finding the *.csv files which are in "/Applications/anaconda/share/gdal"
gdal.SetConfigOption('GDAL_DATA','/Applications/anaconda/share/gdal')
print(gdal.GetConfigOption('GDAL_DATA'))


/Applications/anaconda/share/gdal


### EPSG Codes

- 4326 - WGS84 CRS
- 3310 CA Teale Albers Equal Area
- 2163 US National Atlas Equal Area
- 32160 UTM 10N WGS84


In [29]:
def transformPoint( sourceCRS, targetCRS, x1, y1):

    #input coord order is x/lon , y/lat

    source = osr.SpatialReference()
    source.ImportFromEPSG(sourceCRS)
    #source.ImportFromEPSG(4326) # 4326 is the epsg cood for the WGS84 Geographic Coordinate System 
                                # which is a standard Global CRS


    target = osr.SpatialReference()
    target.ImportFromEPSG(targetCRS)
    #target.ImportFromEPSG(2163) # 2163 is the epsg code for the US National Equal Area projected CRS
                                # see http://spatialreference.org/ref/epsg/2163/

    transform = osr.CoordinateTransformation(source, target)
    the_point = "POINT ("+ str(x1) + " " + str(y1) + ")"
    #print("Input Point: " + the_point)
    point1 = ogr.CreateGeometryFromWkt(the_point)
    #point1 = ogr.CreateGeometryFromWkt("POINT (-122.4061167 47.6149424)")
    point1.Transform(transform) #inplace transformation

    #print ("Output Point: " + point1.ExportToWkt() )
    return point1

In [30]:
# Project our airport points to the US National AEA projection (EPGS Code 2163)
sea_t = transformPoint(4326, 2163, SEA[1], SEA[0])
dfw_t = transformPoint(4326, 2163, DFW[1], DFW[0])
print(dfw_t)
print(sea_t)

POINT (278124.705931125790812 -1338584.79250017250888)
POINT (-1650677.9739719403442 504916.165791458159219)


In [31]:
# We can use Euclidean geometry on projected coordinates
# Calculate euclidean distance using projected coordinates
euclidean_dist(sea_t.GetX(),sea_t.GetY(),dfw_t.GetX(),dfw_t.GetY()) / 1000

2668.103364044094

In [32]:
# Distance calculations for
# two locations along the same latitude- geonames
### Quick & dirty euclidean distance will be pretty accurate bc longitude is constant
stl = (38.743, -90.366) # Lambert–St. Louis International Airport
msy = (29.987, -90.257) # Louis Armstrong New Orleans International Airport 
type(stl[1])
print(euclidean_dist(stl[1],stl[0],msy[1],msy[0]) * 111)
print(haversine_dist(stl[1],stl[0],msy[1],msy[0]))
print(vincenty(stl,msy))

971.991304929
973.673939155
971.352826665 km


In [33]:
# This example shows that distance calculations
# using projected coordinates can employ simple euclidean distance calc and be just as accurate as vincenty.
# (not sure it shows this!)
sfo = (37.619, -122.376)
psp = (33.828, -116.507) #palm springs
print(haversine_dist(sfo[1],sfo[0],psp[1],psp[0]))
print(vincenty(sfo,psp))
psp_t = transformPoint(4326, 3310, psp[1], psp[0])
sfo_t = transformPoint(4326, 3310, sfo[1], sfo[0])
print(euclidean_dist(sfo_t.GetX(),sfo_t.GetY(),psp_t.GetX(),psp_t.GetY()) / 1000)

676.795019567
677.165643669 km
676.990434844


In [34]:
# an aside
import timeit
start = timeit.timeit()
print(euclidean_dist(sfo_t.GetX(),sfo_t.GetY(),psp_t.GetX(),psp_t.GetY()))
end = timeit.timeit()
print (end - start) 

start = timeit.timeit()
print(vincenty(sfo,psp))
end = timeit.timeit()
print (end - start)  

676990.434844
0.00139784812927
677.165643669 km
8.60691070557e-05



## Questions


Elephant seals migrate to Ano Nuevo State Park (37.1436, -122.3472) to mate between the months of Dec and March (http://www.parks.ca.gov/?page_id=1115).
When they depart, males and females go their separate ways. The males travel approximately as far as (56.9643, -153.2407) while the can travel as far as (19.2469, -155.1130). (http://www.esajournals.org/doi/abs/10.1890/0012-9615%282000%29070%5B0353%3AFEONES%5D2.0.CO%3B2)

Questions:

- Do the males or females travel farther?
    - **show your work - indicating how far males and females travel and how you determined those values**
- Are the coordinates above listed as (lat, lon) or (lon,lat)? How can you tell?
- Where do the males & females go respectively? For each group name the closest land mass.


### Notes: questions to add

- try calculating distance between two cities in CA, comparing UTM, CA albers projections to geo results.
- distance between SF0 and London
- can also add folium and map some locations and distances
- add length and area calculations

### Tips

- Keep in mind lat vs lon order in the tuple
- Be mindful of units

### Summary 


- Measuring distance between two points within a small geographic regions can be extremely accurate and fast using projected data. This approach scales up well when working with large amounts of data or many complex spatial calculations.
- For larger distances (and areas), calculations will be more accurate on a spheriod that with a planar map projection, but a bit slower. This is because map projections are designed to minimize distortion within a specifc region - the larger the region the more distortion as you transform space from a spheriod to a plane.


#### From http://postgis.refractions.net/documentation/manual-1.5/ch04.html:

- If your data is contained in a small area, you might find that choosing an appropriate projection and using GEOMETRY is the best solution, in terms of performance and functionality available.
- If your data is global or covers a continental region, you may find that GEOGRAPHY allows you to build a system without having to worry about projection details. You store your data in longitude/latitude, and use the functions that have been defined on GEOGRAPHY.
- If you don't understand projections, and you don't want to learn about them, and you're prepared to accept the limitations in functionality available in GEOGRAPHY, then it might be easier for you to use GEOGRAPHY than GEOMETRY. Simply load your data up as longitude/latitude and go from there.
</pre>